In [14]:
import json

import networkx as nx

In [10]:
with open('searches/data/all_foods.json', mode='r') as f:
    data = json.load(f)
foods = [i['name'].strip() for i in data]
ingredients_foods = {j.strip():[] for i in data for j in i['ingredients']}
for i in data:
    for j in i['ingredients']:
        ingredients_foods[j.strip()].append(i['name'])

<div dir=rtl style="font-size: 30px; font-family: XB Niloofar;">
گراف موادغذایی-غذا
</div>

<div dir=rtl style="font-size: 18px; font-family: XB Niloofar;">
این گراف یک گراف دو بخشی است که یک بخش آن مواد غذایی و بخش دیگر آن غذاها هستند. بین یک غذا و ماده غذایی یال وجود دارد اگر که آن ماده غذایی در غذا به کار رفته باشد.
</div>

In [21]:
G = nx.Graph()
G.add_nodes_from(foods)
G.add_nodes_from(ingredients_foods.keys())

In [22]:
for i, j in ingredients_foods.items():
    for k in j:
        G.add_edge(i, k)

In [23]:
pr = nx.pagerank(G)
with open("searches/data/link1.json", "w", encoding="utf-8") as file:
    link1=sorted([(i, j) for i, j in pr.items()], key=lambda x: x[1], reverse=True)
    json.dump(link1, file)

In [24]:
G = nx.DiGraph()
G.add_nodes_from(foods)
G.add_nodes_from(ingredients_foods.keys())
for i, j in ingredients_foods.items():
    for k in j:
        G.add_edge(k, i)
hubs, authorities = nx.hits(G)

In [26]:
with open("searches/data/link2.json", "w", encoding="utf-8") as file:
    link2=sorted([(i, j) for i, j in hubs.items()], key=lambda x: x[1], reverse=True)
    json.dump(link2, file)

In [20]:
sorted([(i, j) for i, j in authorities.items()], key=lambda x: x[1], reverse=True)[:20]

[('پیاز 1 عدد', 0.021607435394571866),
 ('نمک و فلفل به مقدار لازم', 0.015428953218865773),
 ('روغن به مقدار لازم', 0.01119192274732928),
 ('روغن مایع مقدار کافی', 0.00848881962408101),
 ('زردچوبه به مقدار لازم', 0.008417779870769755),
 ('روغن به میزان لازم', 0.007956660178966127),
 ('پیاز 2 عدد', 0.007817132646301395),
 ('نمک به مقدار لازم', 0.007805287311180096),
 ('نمک و فلفل به میزان لازم', 0.0076326936432505),
 ('سیر 2 حبه', 0.0069840179786986555),
 ('پیاز یک عدد', 0.005519945459536261),
 ('آب به مقدار لازم', 0.005222387095815736),
 ('تخم مرغ 1 عدد', 0.005179734118639716),
 ('تخم مرغ 2 عدد', 0.004923748317575209),
 ('نمک به میزان لازم', 0.004890660874992522),
 ('نمک،فلفل،زردچوبه به مقدار لازم', 0.004757237550071242),
 ('نمک،فلفل،زردچوبه به میزان لازم', 0.0047570639454058775),
 ('پیاز 1عدد', 0.004672552259014872),
 ('نمک فلفل زردچوبه مقدار کافی', 0.004487300728676204),
 ('نمک فلفل مقدار کافی', 0.004106919634678969)]